In [4]:
!pip install openpyxl xlsxwriter pylatexenc scikit-learn
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from pylatexenc.latex2text import LatexNodes2Text, MacroTextSpec

In [5]:
# Load your dataset
train_df = pd.read_csv('ALLtraincompiled.csv')
test_df = pd.read_csv('ALLtestcompiled.csv')

train_df = train_df[['Question','Type']]
test_df = test_df[['Question','Type']]

# Split the dataset into training and test sets
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the label mapping
label_mapping = {'counting_and_probability': 0, 'intermediate_algebra': 1, 'number_theory': 2, 'precalculus':3, 
                'prealgebra':4, 'geometry':5, 'algebra':6}
train_df['Type'] = train_df['Type'].map(label_mapping)
test_df['Type'] = test_df['Type'].map(label_mapping)


In [49]:
# print(train_df['Type'].value_counts())
# print(test_df['Type'].value_counts())
# train_df.head(20)

In [48]:
# def remove_commas_from_latex_numbers(text):
#     # Regular expression to match numbers with commas
#     #pattern = r'(\$[0-9]\d{0,2}(,\!?\d{3})*(\.\d+)?\$)'
#     pattern = r'(\$[0-9]\d{0,2}(,\\?\!?\\?\d{3})*(\.\d+)?\$)'

#     # Function to remove commas from matched numbers
#     def remove_commas(match):
#         return match.group().replace(',', '').replace(r'\!', '')

#     # Replace commas in matched numbers
#     result = re.sub(pattern, remove_commas, text)
#     return result

# # Example input string
# input_text = r"""
# Cedric has deposited $\$12,\!000$ into an account that pays $5\%$ interest compounded annually.

# Daniel has deposited $\$12,\!000$ into an account that pays $7\%$ simple annual interest.

# In $15$ years Cedric and Daniel compare their respective balances. To the nearest dollar, what is the positive difference between their balances?
# """

# # Apply the function to the input string
# output_text = remove_commas_from_latex_numbers(input_text)

# # Print the output string
# print(output_text)


In [44]:
#LatexNodes2Text doesn't work for binomial, combinations, triple dots?
def custom_latex_to_text(latex):
    latex = re.sub(r'\\choose', ' choose', latex)
    latex = re.sub(r'\\binom\{(.*?)\}\{(.*?)\}', r'\1 choose \2', latex)
    latex = re.sub(r'\\dbinom\{(.*?)\}\{(.*?)\}', r'\1 choose \2', latex)
    latex = re.sub(r'\\cdots', '...', latex)
    latex = re.sub(r'\\cdot', '·', latex)                                    # multiplication
    latex = re.sub(r'\\spadesuit', 'spade' , latex)
    latex = re.sub(r'\\clubsuit', 'club', latex)
    latex = re.sub(r'\\diamondsuit', 'diamond' , latex)
    latex = re.sub(r'\\heartsuit', 'heart' , latex)
    latex = re.sub(r'\\dotsm', '· ... ·' , latex)
    latex = re.sub(r'\\left\\lfloor\s*(.*?)\s*\\right\\rfloor', r'floor(\1)', latex)
    #latex = re.sub(r'\\frac\{(.*?)\}\{(.*?)\}', r'\1 / \2', latex)
    latex = re.sub(r'\\tfrac\{(.*?)\}\{(.*?)\}', r'\1 divided by \2', latex)
    latex = re.sub(r'\\dfrac\{(.*?)\}\{(.*?)\}', r'\1 divided by \2', latex)
    #latex = re.sub(r'\\sum_\{(.*?)\}\^\{(.*?)\}', r'sum from \1 to \2 of', latex)
    #latex = re.sub(r'\\sum_\s*\{(.*?)\}\s*\^\s*\{(.*?)\}', r'sum from \1 to \2 of', latex)
    #latex = re.sub(r'\\sum_\s*\{(.*?)\}\s*\^\s*\{(.*?)\}', r'sum from \1 to \2 of', latex)
    #latex = re.sub(r'\\prod_\{(.*?)\}\^\{(.*?)\}', r'product from \1 to \2 of', latex)
    #latex = re.sub(r'\\sum_\s*\{(.*?)\}\s*\^\s*\{(.*?)\}', r'sum from \1 to \2 of', latex) # inc ase that we parsed in \\sum_or also \sum_
    #latex = re.sub(r'\\sum_\{(.*?)\}\s*\^\s*\{(.*?)\}', r'sum from \1 to \2 of', latex)
    latex = re.sub(r'\\sum_?\s*(?:\\limits_?)?\{?(.*?)\}?\s*\^\s*(?:\{?(.*?)\}?\s|(\\infty))', lambda m: 'sum from ' + f'({m.group(1)}' + ' to ' + ('infinity)' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'{m.group(2)})' if m.group(2) else f'{m.group(3)})')) + ' of ', latex)
    latex = re.sub(r'\\sum}_?\s*(?:\\limits_?)?\{?(.*?)\}?\s*\^\s*(?:\{?(.*?)\}?\s|(\\infty))', lambda m: 'sum from ' + f'({m.group(1)}' + ' to ' + ('infinity)' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'{m.group(2)})' if m.group(2) else f'{m.group(3)})')) + ' of ', latex)
    #latex = re.sub(r'\\sum_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'sum from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'(n = {m.group(2)})' if m.group(2) else f'(n = {m.group(3)})')) + ' of ', latex)
    latex = re.sub(r'\\sum}_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'sum from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'(n = {m.group(2)})' if m.group(2) else f'(n = {m.group(3)})')) + ' of ', latex)
    latex = re.sub(r'\\prod_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'product from ' + f'({m.group(1)}' + ' to ' + ('infinity)' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'{m.group(2)})' if m.group(2) else f'{m.group(3)})')) + ' of ', latex)
    latex = re.sub(r'\\prod}_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'product from ' + f'({m.group(1)}' + ' to ' + ('infinity)' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'{m.group(2)})' if m.group(2) else f'{m.group(3)})')) + ' of ', latex)
    #latex = re.sub(r'\\prod_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'product from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'(n = {m.group(2)})' if m.group(2) else f'(n = {m.group(3)})')) + ' of ', latex)
    #latex = re.sub(r'\\prod}_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'product from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'(n = {m.group(2)})' if m.group(2) else f'(n = {m.group(3)})')) + ' of ', latex)
    latex = re.sub(r'(?<!\s)=(?!\s)', ' = ', latex)
    #latex = re.sub(r'\\prod_\s*\{(.*?)\}\s*\^\s*\{(.*?)\}', r'product from \1 to \2 of', latex) # same case as above but \\prod_ or also \prod_
    latex = re.sub(r'\\circ', 'composed with', latex)
    latex = re.sub(r'\\lim\{(.*?)\}', r'lim as \1', latex)
    latex = re.sub(r'\\sin\{(.*?)\}', r'sin(\1)', latex)
    latex = re.sub(r'\\cos\{(.*?)\}', r'cos(\1)', latex)
    latex = re.sub(r'\\log\{(.*?)\}', r'log(\1)', latex)

    return latex

def remove_comma_from_latex(latex_list):
    # Regular expression to match numbers with commas
    #pattern = r'\$([1-9][0-9]{3,}|[1-9][0-9]{5,})'  
    pattern = r'(\$[0-9]\d{0,2}(,\\?\!?\\?\d{3})*(\.\d+)?\$)'
    
    # Function to remove commas from matched numbers
    def remove_comma(match):
        return match.group().replace(',', '')
    
    # Replace commas in numbers >= 1000 for each element in the list
    modified_latex_list = []
    for latex in latex_list:
        modified_latex = re.sub(pattern, remove_comma, latex)
        modified_latex_list.append(modified_latex)
        
    return modified_latex_list
# Test the function with an example
# latex_value = r"$12,\!000$"


# Function to extract LaTeX phrases
def extract_latex(text):
    # Pattern to match $...$, $$...$$, or also \[...\] (all different types of LaTeX for different displaying purposes but our model doesn't care about that)
    # pattern = r'\$(.*?)\$|\$\$(.*?)\$\$|\\\[(.*?)\\\]'
    # matches = re.findall(pattern, text)
    # latex_phrases = [match for groups in matches for match in groups if match] # ugly comprehension, goes through each tuple in the nested tuple the findall gives us and returns the non-empty matches
    # return latex_phrases

    pattern = r'(\$.*?\$)|(\$\$.*?\$\$)|(\\\[.*?\\\])'
    matches = re.findall(pattern, text)
    latex_phrases = [match[0] or match[1] or match[2] for match in matches] 
    return latex_phrases

    #expressions = re.findall(pattern, text, flags=re.DOTALL)
    #return [expr[0] if expr[0] else (expr[1] if expr[1] else expr[2]) for expr in expressions]
    #pattern = r'\$[^\$]*?\$|\$\$[^\$]*?\$\$'
    #return re.findall(pattern, text)

# Function to convert LaTeX to plain text using both LatexNodes2Text and custom 
def convert_latex_to_text(latex_phrases):
    custom_converted = [custom_latex_to_text(phrase) for phrase in latex_phrases]
    # # Convert remaining LaTeX to plain text using LatexNodes2Text
    #custom_converted = [handle_specific_commands(phrase) for phrase in latex_phrases]
    converter = LatexNodes2Text()
    return [converter.latex_to_text(phrase) for phrase in custom_converted]


def replace_latex_with_text(question, latex_phrases, text_phrases):
    for latex, text in zip(latex_phrases, text_phrases):
        question = question.replace(latex, text)
        #question = question.replace(latex, text)
    return question

def remove_newline_characters(text):
    return text.replace('\n', '')

# Extract LaTeX phrases and convert them
# train_df['Question'] = train_df['Question'].apply(remove_comma_from_latex)
train_df['Latex_Phrases'] = train_df['Question'].apply(extract_latex)
train_df['Math_Text'] = train_df['Latex_Phrases'].apply(convert_latex_to_text)
train_df['New_Question'] = train_df.apply(lambda row: replace_latex_with_text(row['Question'], row['Latex_Phrases'], row['Math_Text']), axis=1)

#extra processing to remove some extra newline characters that got added during processing and will mess up the interpretation of the question
train_df["New_Question"] = train_df["New_Question"].apply(remove_newline_characters)

# # Print the DataFrame
#print(train_df[['Question','Latex_Phrases','Math_Text','New_Question']])
train_df.to_csv('questions_with_plain_text.csv', index=False)
train_df.to_excel('questions_with_plain_text.xlsx', index=False, engine='xlsxwriter')

In [47]:
# def extract_latex_4(latex):
#     latex = re.sub(r'\\dbinom\{(.*?)\}\{(.*?)\}', r'\1 choose \2', latex)
#     #latex = re.sub(r'\\sum_\s*\{(.*?)\}\s*\^\s*\{*(.*?)\}*', lambda m: 'sum from ' + m.group(1) + ' to ' + ('infinity' if m.group(2) == '\\infty' else m.group(2)), latex)
#     #latex = re.sub(r'\\sum_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'sum from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'(n = {m.group(2)})' if m.group(2) else f'(n = {m.group(3)})')) + ' of ', latex)
#     #latex = re.sub(r'\\sum_?\s*(\\?)+limits_?\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'sum from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'(n = {m.group(2)})' if m.group(2) else f'(n = {m.group(3)})')) + ' of ', latex)
#     latex = re.sub(r'\\sum_?\s*(?:\\limits_?)?\{?(.*?)\}?\s*\^\s*(?:\{?(.*?)\}?\s|(\\infty))', lambda m: 'sum from ' + f'({m.group(1)}' + ' to ' + ('infinity)' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'{m.group(2)})' if m.group(2) else f'{m.group(3)})')) + ' of ', latex)
#     #latex = re.sub(r'\\sum_(.*?)\s*\^\s*(?:\{(.*?)\}|(.*?))', lambda m: 'sum from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' else (f'({m.group(2)})' if m.group(2) else f'({m.group(3)})')) + ' of ', latex)
#     latex = re.sub(r'\\prod_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'product from ' + f'({m.group(1)}' + ' to ' + ('infinity)' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'{m.group(2)})' if m.group(2) else f'{m.group(3)})')) + ' of ', latex)
#     latex = re.sub(r'\\prod}_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))', lambda m: 'product from ' + f'({m.group(1)}' + ' to ' + ('infinity)' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'{m.group(2)})' if m.group(2) else f'{m.group(3)})')) + ' of ', latex)
#     #pattern = r'\\sum_\s*\{(.*?)\}\s*\^\s*(?:\{(.*?)\}|(\\infty))'
#     #latex = re.sub(pattern, lambda m: 'sum from ' + f'({m.group(1)})' + ' to ' + ('infinity' if m.group(2) == '\\infty' or m.group(3) == '\\infty' else (f'(n = {m.group(2)})' if m.group(2) else f'(n = {m.group(3)})')) + ' of ', latex)
#     return latex

# print(extract_latex_4('\\[\\sum_{n = 1}^\\infty \\frac{2n - 1}{n(n + 1)(n + 2)}.\\]'))
# print(extract_latex_4('\\[\\sum_{n = 1}^{49} \\frac{2n - 1}{n(n + 1)(n + 2)}.\\]'))
# print(extract_latex_4('\\[\\prod_{k = 2}^{100} \\left( \\frac{F_k}{F_{k - 1}} - \\frac{F_k}{F_{k + 1}} \\right) = \\frac{F_a}{F_b}\\]'))
# print(extract_latex_4('\\[\\prod_{k = 2}^\\infty \\left( \\frac{F_k}{F_{k - 1}} - \\frac{F_k}{F_{k + 1}} \\right) = \\frac{F_a}{F_b}\\]'))
# print(extract_latex_4('$P={\\prod}_{j=1}^{12}z_j$'))
# print(extract_latex_4('\\sum_{1 \\le a < b < c} \\frac{1}{2^a 3^b 5^c}.'))
# print(extract_latex_4('S = \\sum \\limits_{k=0}^{62} \\dbinom{2014}{k}'))
# print(extract_latex_4('a_n = \sum\limits_{k=1}^n \sin{k}'))
